###### 2020-11-23 월요일

# 01_텍스트데이터토큰화 & 토큰사전저장

In [14]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import re
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 1. 데이터 불러오기

In [15]:
d1             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data1.csv')
d2             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data2.csv')
d3             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data3.csv', encoding='cp949')
d4             = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data4.xlsx')
d5             = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data5.csv')
d6             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data6.csv')
exist_label_df = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/exist_label_df.csv')

In [16]:
# 모든데이터 하나의 데이터프레임으로 합치기
df_list = [d1, d2, d3, d4, d5, d6, exist_label_df]

all_df = d1
for df in df_list[1:]:
    all_df = pd.concat([all_df, df])

In [17]:
# 필요없는 열 제거
all_df.drop(['Unnamed: 0', '출처'], axis=1, inplace=True)

# 라벨이 NaN인거 제거
index = all_df['악플여부'].isna()
raw_df = all_df[~index]

# 인덱스 리셋
raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [18]:
# 원본유지 복사본 사용
copy_df = raw_df.copy()

In [19]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

##### 악플여부 라벨이 1인경우 제거
 - 악플여부 라벨이 1인경우(악플여부가 애매한 경우)는 사람에 따라 불쾌할수도 있고 아닐수도 있기 때문에 기준을 매기기가 애매하다.
 - 객관적인 기준을 매길수 없기 때문에 라벨이 1인경우 정확도가 떨어지는 현상을 보였다
 - 그래서 라벨이 1인경우를 제거한다

In [20]:
index = copy_df['악플여부'] == 1
copy_df = copy_df[~index]

In [21]:
index = copy_df['악플여부'] == 2
copy_df.loc[index, '악플여부'] = 1

In [22]:
copy_df['악플여부'] = copy_df['악플여부'].apply(int)

In [23]:
copy_df['악플여부'].value_counts()

0    67399
1    14225
Name: 악플여부, dtype: int64

In [24]:
copy_df['댓글'] = copy_df['댓글'].apply(str)

## 2. 토큰화 & 저장
 - Train set, Test set을 나눈다
 - Train set을 기준으로 음절 토큰 사전을 만든다
 - Train set / Test set / 음절토큰사전을 저장한다
    - 이렇게 저장하는 이유는? : 매번 음절토큰사전을 만드는데 시간이 많이 소요되기 때문에 한번 만들어놓고 재사용하여 시간을 절약!


In [25]:
def tokenization_save(df):

    feature = df['댓글']
    label = df['악플여부']

    feature_split = feature.apply(list).tolist()
    feature_token_list = sum(feature_split, [])
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(feature_token_list)
    
    word_index_vocab = tokenizer.word_index

    return feature, label, word_index_vocab


In [26]:
def save_feature_target(feature, target) :
    save_df_list = [feature, target]
    file_names = ['feature', 'target']

    for idx, set in enumerate(save_df_list):
        set.to_csv('/content/drive/MyDrive/[final_project]_악플원정대/02_K-Fold_O/data/' + file_names[idx] + '.csv', encoding='UTF-8-SIG')


def save_word_index(word_index_vocab) :
    json_file = json.dumps(word_index_vocab, ensure_ascii=False)
    f = open('/content/drive/MyDrive/[final_project]_악플원정대/02_K-Fold_O/data/word_index_vocab.json', 'w')
    f.write(json_file)
    f.close()

In [27]:
feature, target, word_index_vocab = tokenization_save(copy_df)

In [28]:
# ' '(공백)도 사전에 포함 
word_index_vocab[' '] = len(word_index_vocab) + 1

In [30]:
save_feature_target(feature, target)
save_word_index(word_index_vocab)